# Mini-Project 2
## Part 1: Directly learning vector embeddings for users and movies

### Import libraries
- pandas for constructing train and test dataframes
- numpy for mathematical operations on arrays
- must use pytorch (torch) when developing code for this project
- torch.nn for defining our model
- train_test_split from sklearn.model_selection to *construct* train and test dataframes

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

### Construct train and test dataframes
Split data so that 70% is used for train and 30% for test.

Sort values by time, in ascending order. According to the assignment document:

**A common thing to do is to use the older data as the training set and the newer data as the test/validation sets. That way, you get a good estimate for whether your algorithm would be accurate on new data.**

In [ ]:
ratings = pd.read_csv("http://www.cs.toronto.edu/~guerzhoy/324/movielens/ratings.csv")
ratings.sort_values(by=['timestamp'], ascending=True)
train_df, test_df = train_test_split(ratings, test_size=0.3)

### Map each movie id to an index.

There are many movies, but they are not represented as integers from 0 to (number of movies - 1). So, we store a one-to-one mapping from movie id to idx and idx to movie id. To do this, we extract an array of unique movie ids and map each movie id to its index in the array.

In [ ]:
movieids = ratings.movieId.unique()
movieid_to_idx = dict()
idx_to_movieid = dict() 
for i in range(len(movieids)):
    movieid_to_idx[movieids[i]] = i
    idx_to_movieid[i] = movieids[i]

### Set n and d
n is computed, it is the total number of items to embed. Since we want embeddings for users and movies, the equation to compute n is n = n_users + n_movies

d is the dimension of the embeddings. We set it to 10.

In [ ]:
n_users, n_movies = ratings.nunique()['userId'], ratings.nunique()['movieId']

n = n_users + n_movies

d = 10

device = 'cuda'

### Define function for obtaining corresponding movie index from id
This will be used in the next cell, to map a list of ids to indices

In [ ]:
def getmovieidx(movieid):
    return movieid_to_idx[movieid]

### Create adjacency matrix for train and test sets
To do this, first initialize train and test matrices with dimensions (num users, num movies). 

Each row corresponds to a user, and "1" entries in a row correspond to movies that a user has given 5 stars to. 

In [ ]:
shape = (n_users, n_movies)

adjacency_matrix_train = torch.zeros(shape).to(device)
adjacency_matrix_test = torch.zeros(shape).to(device)

unique_users = ratings.userId.unique()

for user_id in unique_users:
    users_5star_movies_train = train_df[train_df['userId']==user_id][train_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_train)
    users_5star_movies_train = [item for item in x]
    adjacency_matrix_train[user_id-1][users_5star_movies_train] = 1

    users_5star_movies_test = test_df[test_df['userId']==user_id][test_df['rating']==5.0].movieId.unique()
    y = map(getmovieidx, users_5star_movies_test)
    users_5star_movies_test = [item for item in y]
    adjacency_matrix_test[user_id-1][users_5star_movies_test] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


### Define train loop function, which takes model and optimizer, and updates matrix of embeddings to improve the objective function defined in the assignment handout.

Note that optimizing the objective function is same as minimizing the negative of the objective function. So, we define the loss as negative of the objective function.

To calculate the loss, we iterate over users. For each user we find the product between its embedding and embedding of all the movies he/she rated 5 stars, as well as vector-matrix product between user embedding and embeddings not rated 5 stars. We then add the "good movie" dot products and subtract the "bad movie" dot products, over all users, while upweighting the "good movies" by 200, to emphasize that this is the relationship we want to learn.

In [ ]:
def train_loop(model, optimizer):
    loss = 0
    for user_idx in range(n_users):
        loss = 0
        user_emb = model(torch.tensor(user_idx).to(device)).to(device)
        
        users_5star_movieindices = torch.where(adjacency_matrix_train[user_idx]==1)[0].to(device)
      
        good_rows = users_5star_movieindices + n_users
        good_embeddings = model(good_rows).to(device)
        
        bad_movieindices = torch.where(adjacency_matrix_train[user_idx]==0)[0]
        bad_rows = bad_movieindices + n_users
        bad_embeddings = model(bad_rows).to(device)
        
        good_dps = torch.matmul(user_emb, good_embeddings.T).to(device)
        bad_dps = torch.matmul(user_emb, bad_embeddings.T).to(device)
        
        good_sigmoids = torch.sigmoid(good_dps).to(device)
        bad_sigmoids = torch.sigmoid(-1*bad_dps).to(device)
        
        good_logsigs = torch.log10(good_sigmoids).to(device)
        bad_logsigs = torch.log10(bad_sigmoids).to(device)
        loss += (200*torch.sum(good_logsigs) + torch.sum(bad_logsigs))
        loss*=-1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss)

### Recall150 computation

In [ ]:
def recall150 (adj_mat, model): # pass in true adj_mat, will compare with predicted here
    with torch.no_grad():
        user_mat = model.embedding.weight[:n_users, :]
        movie_mat = model.embedding.weight[n_users:, :]
        scores = torch.matmul(user_mat, movie_mat.T).to(device)

        for i in range(scores.shape[0]):
          split_val = torch.quantile(scores[i], 1-(150/n_movies))
          mask0 = torch.where(scores[i] < split_val)[0]
          mask1 = torch.where(scores[i] >= split_val)[0]
          scores[i][mask0] = 0
          scores[i][mask1] = 1 
          
        #compare
        avg = 0
        valid = 0
        for i in range(scores.shape[0]):
          a1 = torch.where(adj_mat[i] == 1)[0]
          a2 = torch.where(scores[i] == 1)[0]
          
          a1=a1.cpu().detach().numpy()
          a2=a2.cpu().detach().numpy()

          x = len(np.intersect1d(a1, a2))
          y = len(torch.where(adj_mat[i] == 1)[0])       

          if y != 0:                           
            avg += x/y
            valid += 1
        avg = avg/valid
        return avg

### Define neural network.
We want to optimize the parameters of our neural network. Our network for this project is just one embedding layer. The parameters of the embedding layer comprises of the embeddings for each user and movie (corresponding to rows). Initialize weights to be uniform from -1 to 1, which empirically gave better results.

In [ ]:
class EmbeddingLayer(nn.Module):
    def __init__(self):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(n, d)
        self.embedding.weight.data.uniform_(-1, 1)

    def forward(self, x):
        x = self.embedding(x)
        return x

model = EmbeddingLayer().to(device)
print(f"Model structure: {model}\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: EmbeddingLayer(
  (embedding): Embedding(10334, 10)
)

Layer: embedding.weight | Size: torch.Size([10334, 10]) | Values : tensor([[-0.8885, -0.3680,  0.0241, -0.2537,  0.6816, -0.2937, -0.6296, -0.1478,
         -0.0261,  0.9802],
        [ 0.4553, -0.5951,  0.9131,  0.3119, -0.6157,  0.1831, -0.3336,  0.2888,
         -0.2130,  0.4221]], device='cuda:0', grad_fn=<SliceBackward0>) 



### Execute training
Train for 20 epochs, with a standard learning rate of 0.01. Empirically, Adam optimizer exhibits better convergence behaviour than stochastic gradient descent, so it is our choice of optimizer. 

We print the recall scores for both training and test sets after each epoch

In [ ]:
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, optimizer)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_test, model)}")
print("Done!")

Epoch 1
-------------------------------
tensor(10906.3740, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.17723131909443607
Validation Recall150:
0.08219185574691062
Epoch 2
-------------------------------
tensor(9430.2930, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.3219781891770597
Validation Recall150:
0.11678377694650956
Epoch 3
-------------------------------
tensor(7173.6709, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.5169751815207138
Validation Recall150:
0.2966030063446902
Epoch 4
-------------------------------
tensor(4582.0391, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.6556202008382463
Validation Recall150:
0.36794174050806017
Epoch 5
-------------------------------
tensor(3606.9429, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.7221589681762232
Validation Recall150:
0.39028151389519405
Epoch 6
-------------------------------
tensor(2145.4612, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150

### Compute recall150 score on test set
The recall150 score on test set is approximately 40%. Since there are no hyperparameter tuning here, our test set is also used as a validation set in the above training.

In [ ]:
print(f"Test Set Recall\n{recall150(adjacency_matrix_test, model)}")

Test Set Recall
0.39892624854606684


# Mini Project 2
## Part 2
The requirements for this part of the Mini Project are pasted below:

**Learning embeddings using Node2Vec\
Implement a learning procedure for Node2Vec embeddings.**

**Report Recall@150 for the training and test sets, for the task of predicting which movies a given user will give 5 stars to.**

**Report what you had done in order to improve the performance of Node2Vec over the initial parameters that you had tried. You only need to demonstrate progress in improving Node2Vec, and do not need to beat the performance of embeddings learned directly."**

### Import Libraries
- pandas for extracting dataframes: train, val, test
- numpy for mathematical operations on arrays.
- must use PyTorch when developing the code for this mini-project
- import torch.nn to easily implement neural networks.
- train_test_split form sklearn.model_selection to split dataset into train, val and test sets.


In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

### Construct train, validation and test dataframes
Split data so that 70% is used for train, 20% for validation and 10% for test set.

Sort values by time, in ascending order. According to the assignment document:

**A common thing to do is to use the older data as the training set and the newer data as the test/validation sets. That way, you get a good estimate for whether your algorithm would be accurate on new data.**

In [12]:
ratings = pd.read_csv("http://www.cs.toronto.edu/~guerzhoy/324/movielens/ratings.csv")
ratings.tail()
ratings.sort_values(by=['timestamp'], ascending=True)
train_df, test_val_df = train_test_split(ratings, test_size=0.7)
val_df, test_df = train_test_split(test_val_df, test_size=0.333)

### Map each movie id to an index.

There are many movies, but they are not represented as integers from 0 to (number of movies - 1). So, we store a one-to-one mapping from movie id to idx and idx to movie id. To do this, we extract an array of unique movie ids and map each movie id to its index in the array.

In [13]:
movieids = ratings.movieId.unique()
movieid_to_idx = dict()
idx_to_movieid = dict()
for i in range(len(movieids)):
    movieid_to_idx[movieids[i]] = i
    idx_to_movieid[i] = movieids[i]

### Set n and d
n is computed, it is the total number of items to embed. Since we want embeddings for users and movies, the equation to compute n is n = n_users + n_movies

d is the dimension of the embeddings. We set it to 10.

In [14]:
n_users, n_movies = ratings.nunique()['userId'], ratings.nunique()['movieId']

n = n_users + n_movies

d = 10

device = 'cuda'

### Define function for obtaining corresponding movie index from id
This will be used in the next cell, to map a list of ids to indices

In [15]:
def getmovieidx(movieid):
    return movieid_to_idx[movieid]

### Create adjacency matrix for train, validation and test sets
To do this, first initialize train, validation and test matrices with dimensions (num users, num movies). 

Each row corresponds to a user, and "1" entries in a row correspond to movies that a user has given 5 stars to. 

In [16]:
shape = (n_users, n_movies)

adjacency_matrix_train = torch.zeros(shape).to(device)
adjacency_matrix_val = torch.zeros(shape).to(device)
adjacency_matrix_test = torch.zeros(shape).to(device)

unique_users = ratings.userId.unique()

for user_id in unique_users:
    users_5star_movies_train = train_df[train_df['userId']==user_id][train_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_train)
    users_5star_movies_train = [item for item in x]
    adjacency_matrix_train[user_id-1][users_5star_movies_train] = 1

    users_5star_movies_val = val_df[val_df['userId']==user_id][val_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_val)
    users_5star_movies_val = [item for item in x]
    adjacency_matrix_val[user_id-1][users_5star_movies_val] = 1

    users_5star_movies_test = test_df[test_df['userId']==user_id][test_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_test)
    users_5star_movies_test = [item for item in x]
    adjacency_matrix_test[user_id-1][users_5star_movies_test] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Recall150 computation

In [17]:
def recall150 (adj_mat, model): # pass in true adj_mat, will compare with predicted here
    with torch.no_grad():
        user_mat = model.embedding.weight[:n_users, :]
        movie_mat = model.embedding.weight[n_users:, :]
        scores = torch.matmul(user_mat, movie_mat.T).to(device)

        for i in range(scores.shape[0]):
          split_val = torch.quantile(scores[i], 1-(150/n_movies))
          mask0 = torch.where(scores[i] < split_val)[0]
          mask1 = torch.where(scores[i] >= split_val)[0]
          scores[i][mask0] = 0
          scores[i][mask1] = 1 
          
        #compare
        avg = 0
        valid = 0
        for i in range(scores.shape[0]):
          a1 = torch.where(adj_mat[i] == 1)[0]
          a2 = torch.where(scores[i] == 1)[0]
          # print(a1)
          # print(a2)
          a1=a1.cpu().detach().numpy()
          a2=a2.cpu().detach().numpy()
          x = len(np.intersect1d(a1, a2))
          y = len(torch.where(adj_mat[i] == 1)[0])       
          # print(f"x: {x}")
          # print(f"y: {y}")
          if y != 0:                           
            avg += x/y
            valid += 1
        avg = avg/valid
        return avg

### Compute and store degrees of each node. 
This will be used in negative sampling, when sampling K random nodes that are not part of random walk.

In [18]:
# Compute degree probabilities
# we must use train set
user_to_movie = adjacency_matrix_train.clone()
movie_to_user = adjacency_matrix_train.T
degrees = []
for user_idx in range(user_to_movie.shape[0]):
  degrees.append(len(torch.where(user_to_movie[user_idx] == 1)[0]))
for movie_idx in range(movie_to_user.shape[0]):
  degrees.append(len(torch.where(movie_to_user[movie_idx]==1)[0]))

In [19]:
# x = torch.zeros(5,5)
# x[0][torch.tensor([2,3]) + 1] = 1
# x

### Construct num_nodes * num_nodes adjacency matrix.
We do not want to distinguish between users and movies. So, treat each as node. Then there is n = num_users + num_movies nodes, and adjacency matrix is n by n

In [20]:
num_nodes = n_users+n_movies
adj_mat = torch.zeros(num_nodes, num_nodes)
for user_idx in range(n_users):
  adj_mat[user_idx][torch.where(user_to_movie[user_idx] == 1)[0] + n_users] = 1
for movie_idx in range(n_movies):
  adj_mat[n_users+movie_idx][torch.where(movie_to_user[movie_idx]==1)[0]] = 1

### Specify unnormalized transition probability
For each node, specify transition probability to neighbours as 1/q and non-neighbours to 0.

When we conduct our random walk, when determining transition probabilities, we just use these pre-computed probabilities, plus one extra step of setting probability of previous node to 1/p.

In [21]:
def compute_transition_probs(q):
  transition_prob = torch.zeros(num_nodes, num_nodes)
  for node_idx in range(num_nodes):
    # print(node_idx)
    transition_prob[node_idx] = torch.tensor([0]*num_nodes, dtype=torch.float64).to(device)
    valid_nodes = torch.where(adj_mat[node_idx] == 1)[0]
    transition_prob[node_idx][valid_nodes] = 1/q
  return transition_prob

transition_probs = compute_transition_probs(q=3)

In [ ]:
# x = torch.tensor([3,4,5])
# y = x.clone()
# y[1] = 3
# print(x)
# print(y)

### Define next_node function which given current node, previous node, transition probabilities and parameter p, selects next node in random walk. 
This is done by indexing into transition_probs, setting previous node probability to 1/p (if it exists, i.e. current node is not first node in walk) and randomly choosing the next node with the computed probabilities.

In [22]:
def next_node(previous, current, p, q):
  node_indices = list(range(num_nodes))
  node_probs = transition_probs[current].clone()
  if previous != -1:
    node_probs[previous] = 1/p
  chosen_user = random.choices(node_indices, weights=node_probs, k=1)
  return chosen_user[0]

### Define random_walk function, which outputs an array of nodes corresponding to random walk starting from start node and taking num_steps steps.

In [23]:
import random
def random_walk(start_node, num_steps, p, q):
  walk = [start_node]
  while len(walk) < num_steps:
    current = walk[-1]
    previous = walk[-2] if len(walk) > 1 else -1
    next = next_node(previous, current, p, q)
    walk.append(next)
  return walk

In [ ]:
# x = torch.zeros(2, 3, 5)
# x[1,1] = torch.tensor([3,4,5,6,7])
# x

### Simulate random walks 
We use the function random_walk above to generate a number of random walks (num_walks) starting from every node and taking a number of steps (num_steps)

In [24]:
def simulate_walks(num_walks, num_steps, p, q):
  cache = torch.zeros(num_nodes, num_walks, num_steps)
  for node_idx in range(num_nodes):
    print(node_idx)
    for walk_count in range(num_walks):
      cache[node_idx, walk_count] = torch.tensor(random_walk(node_idx, num_steps, p, q))
  return cache
p = 1
q = 3
num_walks = 20
num_steps = 2
cache = simulate_walks(num_walks, num_steps, p, q)

### Define function to construct adjacency matrix given cache of walks
This function will add an edge to our graph in row r, column c if there is at least one time where we go from node r to node c when we start our random walk from node r.

In [26]:
def construct_adj_mat(cache, num_walks):
  adj_mat = torch.zeros(num_nodes, num_nodes)
  for node_idx in range(num_nodes):
    # print(node_idx)
    for walk in cache[node_idx]:
      adj_mat[node_idx][walk.long()] = 1
  return adj_mat                      
random_walk_adj_mat = construct_adj_mat(cache, 1)

### Define train loop function.
In one iteration of the train cycle, we will iterate over all nodes, and compute dot product between this node's embedding and all nodes it has an edge with. Find the sum of these and subtract dot product between this embedding and K randomly sampled nodes form the negative set. Then compute loss, upweighting positive samples by a parameter upweight and step through optimizer to update weights. 

In [27]:
def train_loop(model, optimizer, random_walk_adj_mat, K, upweight):
    loss = 0
    for node_idx in range(num_nodes):
        loss = 0
        node_emb = model(torch.tensor(node_idx).to(device)).to(device)
        
        good_nodes = torch.where(random_walk_adj_mat[node_idx]==1)[0].to(device)
        good_embeddings = model(good_nodes).to(device)
        
        bad_nodes = torch.where(random_walk_adj_mat[node_idx]==0)[0]
        probs = torch.tensor(degrees)[bad_nodes]
        neg_samples = random.choices(bad_nodes.tolist(), weights=probs.tolist(), k=K)
        neg_samples = torch.tensor(neg_samples).to(device)
        bad_embeddings = model(neg_samples).to(device)
        
        good_dps = torch.matmul(node_emb, good_embeddings.T).to(device)
        bad_dps = torch.matmul(node_emb, bad_embeddings.T).to(device)
        
        good_sigmoids = torch.sigmoid(good_dps).to(device)
        bad_sigmoids = torch.sigmoid(-1*bad_dps).to(device)
        
        good_logsigs = torch.log10(good_sigmoids).to(device)
        bad_logsigs = torch.log10(bad_sigmoids).to(device)
        loss += (upweight*torch.sum(good_logsigs) + torch.sum(bad_logsigs))
        loss*=-1
        optimizer.zero_grad()
        loss.backward()
        # print(model.embedding.weight.grad)
        optimizer.step()
    print(loss)

### Define neural network.
We want to optimize the parameters of our neural network. Our network for this project is just one embedding layer. The parameters of the embedding layer comprises of the embeddings for each user and movie (corresponding to rows). Initialize weights to be uniform from -1 to 1, which empirically gave better results.

In [28]:
class EmbeddingLayer(nn.Module):
    def __init__(self):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(n, d)
        self.embedding.weight.data.uniform_(-1, 1)

    def forward(self, x):
        x = self.embedding(x)
        return x

# model = EmbeddingLayer().to(device)
# print(f"Model structure: {model}\n")

# for name, param in model.named_parameters():
#     print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

## Model Type 1
p = 1

q = 3

num_walks = 1

num_steps = 5

Model 1:
- learning rate = 1e-3
- K = 2
- upweight = 200

In [ ]:
model1 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer = torch.optim.SGD(model1.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K = 2 # num_steps = 5 but bfs more likely than dfs, so set K=2 to balance. we expect num unique nodes in each walk to be ~2.
upweight = 200

epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model1, optimizer, random_walk_adj_mat, K, upweight)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model1)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model1)}")
print("Done!")

Epoch 1
-------------------------------
tensor(29.7941, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.015309075846569759
Validation Recall150:
0.01630882414056124
Epoch 2
-------------------------------
tensor(24.3747, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.018607089416452026
Validation Recall150:
0.01538764656106078
Epoch 3
-------------------------------
tensor(18.7875, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.028123469333001982
Validation Recall150:
0.015808284123685245
Epoch 4
-------------------------------
tensor(12.6127, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.04187896112927905
Validation Recall150:
0.02040976354552965
Epoch 5
-------------------------------
tensor(7.7656, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.06612056889002177
Validation Recall150:
0.02383818903798698
Epoch 6
-------------------------------
tensor(4.5122, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.08

Model 2:
- learning_rate = 1e-3
- K = 2
- upweight = 10

In [ ]:
model2 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer2 = torch.optim.SGD(model2.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K2 = 2 # num_steps = 5 but bfs more likely than dfs, so set K=2 to balance. we expect num unique nodes in each walk to be ~2.
upweight2 = 10 # make effect of upweight smaller to see how much it effects results

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model2, optimizer2, random_walk_adj_mat, K2, upweight2)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model2)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model2)}")
print("Done!")

Epoch 1
-------------------------------
tensor(3.5892, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.008556444024088188
Validation Recall150:
0.010408862574587304
Epoch 2
-------------------------------
tensor(3.5774, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.008556444024088188
Validation Recall150:
0.009936392657672943
Epoch 3
-------------------------------
tensor(3.5287, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.008556444024088188
Validation Recall150:
0.009793942515222802
Epoch 4
-------------------------------
tensor(3.5039, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.008240188489616336
Validation Recall150:
0.009793942515222802
Epoch 5
-------------------------------
tensor(3.5962, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.008113686275827594
Validation Recall150:
0.009793942515222802
Epoch 6
-------------------------------
tensor(3.5275, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.

Model 3:
- learning_rate = 1e-3
- K = 2
- upweight = 500

In [ ]:
model3 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer3 = torch.optim.SGD(model3.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K3 = 2 # num_steps = 5 but bfs more likely than dfs, so set K=2 to balance. we expect num unique nodes in each walk to be ~2.
upweight3 = 500 # make effect of upweight smaller to see how much it effects results

epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model3, optimizer3, random_walk_adj_mat, K3, upweight3)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model3)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model3)}")
print("Done!")

Epoch 1
-------------------------------
tensor(34.0162, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.028162039657716943
Validation Recall150:
0.010829642347159569
Epoch 2
-------------------------------
tensor(4.9577, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.06130597292056377
Validation Recall150:
0.017358414783591944
Epoch 3
-------------------------------
tensor(0.7782, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.14371878812532152
Validation Recall150:
0.02628540680001843
Epoch 4
-------------------------------
tensor(0.7584, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.21226260097922167
Validation Recall150:
0.033490740051310114
Epoch 5
-------------------------------
tensor(2.3832, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.2629910995681937
Validation Recall150:
0.04004396060898495
Epoch 6
-------------------------------
tensor(0.0110, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.278749

# Model Type 2
p = 1

q = 3

num_walks = 10

num_steps = 2

### Simulate new walks 
Simulate walks given new parameters

In [ ]:
p = 1
q = 3
num_walks = 10
num_steps = 2
cache = simulate_walks(num_walks, num_steps, p, q)

### Construct new adjacency matrix from newly simulated walks

In [30]:
random_walk_adj_mat = construct_adj_mat(cache, num_walks)

Model 4:
- learning_rate = 1e-3
- K = 5
- upweight = 200

In [ ]:
model4 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer4 = torch.optim.SGD(model4.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K4 = 5 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight4 = 200

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model4, optimizer4, random_walk_adj_mat, K4, upweight4)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model4)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model4)}")
print("Done!")

Epoch 1
-------------------------------
tensor(49.7420, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.02076493060323217
Validation Recall150:
0.011143133158280815
Epoch 2
-------------------------------
tensor(46.9331, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.029734736676168022
Validation Recall150:
0.011348732726137015
Epoch 3
-------------------------------
tensor(42.5829, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.036448329414940396
Validation Recall150:
0.012988991849530347
Epoch 4
-------------------------------
tensor(35.4418, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.04765931215194141
Validation Recall150:
0.01647652226220962
Epoch 5
-------------------------------
tensor(26.8021, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.07394034492113699
Validation Recall150:
0.02015453547171135
Epoch 6
-------------------------------
tensor(17.1767, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0

**Try both smaller (100) and larger(400) upweight**

Model 5:
- learning_rate = 1e-3
- K = 5
- upweight = 100

In [33]:
model5 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer5 = torch.optim.SGD(model5.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K5 = 5 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight5 = 100

epochs = 45
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model5, optimizer5, random_walk_adj_mat, K5, upweight5)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model5)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model5)}")
print("Done!")

# Re-run, maybe # epochs was small
# This was run 30+15=45 epochs.

Epoch 1
-------------------------------
tensor(28.1396, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.017944736204583485
Validation Recall150:
0.015428908666489272
Epoch 2
-------------------------------
tensor(27.6315, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.01860620115714949
Validation Recall150:
0.01402337585512311
Epoch 3
-------------------------------
tensor(26.6234, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.022318760931470005
Validation Recall150:
0.013454840315130445
Epoch 4
-------------------------------
tensor(25.5477, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.027299356088670103
Validation Recall150:
0.013022944065092024
Epoch 5
-------------------------------
tensor(24.3358, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.03135569137997938
Validation Recall150:
0.013123428894843877
Epoch 6
-------------------------------
tensor(23.4570, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:

Model 6:
- learning_rate = 1e-3
- K = 5
- upweight = 400

In [ ]:
model6 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer6 = torch.optim.SGD(model6.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K6 = 5 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight6 = 400

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model6, optimizer6, random_walk_adj_mat, K6, upweight6)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model6)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model6)}")
print("Done!")

Epoch 1
-------------------------------
tensor(53.0327, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.019640935153208812
Validation Recall150:
0.012640035315508967
Epoch 2
-------------------------------
tensor(12.3447, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.04578512887440081
Validation Recall150:
0.017284580245223415
Epoch 3
-------------------------------
tensor(2.7393, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.10266944597330674
Validation Recall150:
0.02667594753397782
Epoch 4
-------------------------------
tensor(2.2469, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.17743502973293088
Validation Recall150:
0.03504768360643473
Epoch 5
-------------------------------
tensor(2.0267, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.25561497278906525
Validation Recall150:
0.04613507281004937
Epoch 6
-------------------------------
tensor(14.3310, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.3091

## Model Type 3
p = 1

q = 3

num_walks = 20

num_steps = 2

### Simulate new walks 
Simulate walks given new parameters

In [ ]:
p = 1
q = 3
num_walks = 20
num_steps = 2
cache = simulate_walks(num_walks, num_steps, p, q)

### Construct new adjacency matrix from newly simulated walks

In [ ]:
random_walk_adj_mat = construct_adj_mat(cache, num_walks)

Model 7: 
- learning_rate = 1e-3
- K = 5
- upweight = 200

In [ ]:
model7 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer7 = torch.optim.SGD(model7.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K7 = 5 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight7 = 200

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model7, optimizer7, random_walk_adj_mat, K7, upweight7)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model7)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model7)}")
print("Done!")

Epoch 1
-------------------------------
tensor(48.6338, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.013754556051282271
Validation Recall150:
0.012050598250403818
Epoch 2
-------------------------------
tensor(45.8796, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.025229244982661596
Validation Recall150:
0.012275480807054234
Epoch 3
-------------------------------
tensor(42.3036, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.046111898431915384
Validation Recall150:
0.01751374818953083
Epoch 4
-------------------------------
tensor(35.7005, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.07482766229438896
Validation Recall150:
0.024955647683810582
Epoch 5
-------------------------------
tensor(28.3742, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.10390821761763924
Validation Recall150:
0.03308158516679714
Epoch 6
-------------------------------
tensor(18.3202, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:


Model 8: 
- learning_rate = 1e-3
- K = 5
- upweight = 300

In [ ]:
model8 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer8 = torch.optim.SGD(model8.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K8 = 5 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight8 = 300

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model8, optimizer8, random_walk_adj_mat, K8, upweight8)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model8)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model8)}")
print("Done!")

Epoch 1
-------------------------------
tensor(46.9720, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.021985608575386274
Validation Recall150:
0.011617457541271954
Epoch 2
-------------------------------
tensor(35.5141, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.04826058135514937
Validation Recall150:
0.020062115910343157
Epoch 3
-------------------------------
tensor(21.0061, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.09756700589314687
Validation Recall150:
0.027645978490126066
Epoch 4
-------------------------------
tensor(6.4385, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.14995824526909643
Validation Recall150:
0.041084087223031786
Epoch 5
-------------------------------
tensor(2.4223, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.20053691896892123
Validation Recall150:
0.05252154876496808
Epoch 6
-------------------------------
tensor(6.2183, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.27

Model 9: 
- learning_rate = 1e-3
- K = 10
- upweight = 200

In [ ]:
model9 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer9 = torch.optim.SGD(model9.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K9 = 10 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight9 = 200

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model9, optimizer9, random_walk_adj_mat, K9, upweight9)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model9)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model9)}")
print("Done!")

Epoch 1
-------------------------------
tensor(40.7949, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.012479108101009594
Validation Recall150:
0.017760384224497192
Epoch 2
-------------------------------
tensor(36.4532, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.01990681550026628
Validation Recall150:
0.018504528301754905
Epoch 3
-------------------------------
tensor(30.1059, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.03322694054595516
Validation Recall150:
0.02145940983233093
Epoch 4
-------------------------------
tensor(20.8061, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.0604136064081355
Validation Recall150:
0.024730918212162875
Epoch 5
-------------------------------
tensor(11.6288, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.10023894724314654
Validation Recall150:
0.031843333258280884
Epoch 6
-------------------------------
tensor(6.3975, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.1

Model 10: 
- learning_rate = 1e-3
- K = 10
- upweight = 300

In [ ]:
model10 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer10 = torch.optim.SGD(model10.parameters(), lr=learning_rate) # We prefer Adam but instructions say to use SGD for part 2.
K10 = 10 # num_walks = 10 but bfs more likely than dfs, so set K=5 to balance. we expect num unique nodes in each walk to be ~5.
upweight10 = 300

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model10, optimizer10, random_walk_adj_mat, K10, upweight10)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model10)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model10)}")
print("Done!")

Epoch 1
-------------------------------
tensor(52.0658, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.020543273593647958
Validation Recall150:
0.017306057223256316
Epoch 2
-------------------------------
tensor(31.1491, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.04556682452724549
Validation Recall150:
0.023081524480345172
Epoch 3
-------------------------------
tensor(11.9821, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.0862242628851104
Validation Recall150:
0.03495502710244859
Epoch 4
-------------------------------
tensor(4.4194, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.16005871202118852
Validation Recall150:
0.050376570384268536
Epoch 5
-------------------------------
tensor(7.8130, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.22817278308935438
Validation Recall150:
0.062326508656627796
Epoch 6
-------------------------------
tensor(11.7393, device='cuda:0', grad_fn=<MulBackward0>)
Train Recall150:
0.28

**Model 8 performs the best with respect to validation set. So, we will select it as our best model. Let's evaluate its accuracy on the test set.**

In [ ]:
print(f"Test Recall150:\bn{recall150(adjacency_matrix_test, model8)}")

Test Recall150:
0.10424172843355602


The result we have obtained matches with the prof and other students, according to Piazza posts. So, we end our hyperparameter tuning for this training approach.

## Training Approach #2
We change our training strategy in an attempt to get better results than 10% on the test set. 

Same as before

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

Same as before

In [3]:
ratings = pd.read_csv("http://www.cs.toronto.edu/~guerzhoy/324/movielens/ratings.csv")
ratings.tail()
ratings.sort_values(by=['timestamp'], ascending=True)
train_df, test_val_df = train_test_split(ratings, test_size=0.7)
val_df, test_df = train_test_split(test_val_df, test_size=0.333)

Same as before

In [4]:
movieids = ratings.movieId.unique()
movieid_to_idx = dict()
idx_to_movieid = dict()
for i in range(len(movieids)):
    movieid_to_idx[movieids[i]] = i
    idx_to_movieid[i] = movieids[i]

Same as before

In [5]:
n_users, n_movies = ratings.nunique()['userId'], ratings.nunique()['movieId']

n = n_users + n_movies

d = 10

device = 'cuda'

Same as before

In [6]:
def getmovieidx(movieid):
    return movieid_to_idx[movieid]

Same as before

In [7]:
shape = (n_users, n_movies)

adjacency_matrix_train = torch.zeros(shape).to(device)
adjacency_matrix_val = torch.zeros(shape).to(device)
adjacency_matrix_test = torch.zeros(shape).to(device)

unique_users = ratings.userId.unique()

for user_id in unique_users:
    users_5star_movies_train = train_df[train_df['userId']==user_id][train_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_train)
    users_5star_movies_train = [item for item in x]
    adjacency_matrix_train[user_id-1][users_5star_movies_train] = 1

    users_5star_movies_val = val_df[val_df['userId']==user_id][val_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_val)
    users_5star_movies_val = [item for item in x]
    adjacency_matrix_val[user_id-1][users_5star_movies_val] = 1

    users_5star_movies_test = test_df[test_df['userId']==user_id][test_df['rating']==5.0].movieId.unique()
    x = map(getmovieidx, users_5star_movies_test)
    users_5star_movies_test = [item for item in x]
    adjacency_matrix_test[user_id-1][users_5star_movies_test] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Same as before

In [8]:
def recall150 (adj_mat, model): # pass in true adj_mat, will compare with predicted here
    with torch.no_grad():
        user_mat = model.embedding.weight[:n_users, :]
        movie_mat = model.embedding.weight[n_users:, :]
        scores = torch.matmul(user_mat, movie_mat.T).to(device)

        for i in range(scores.shape[0]):
          split_val = torch.quantile(scores[i], 1-(150/n_movies))
          mask0 = torch.where(scores[i] < split_val)[0]
          mask1 = torch.where(scores[i] >= split_val)[0]
          scores[i][mask0] = 0
          scores[i][mask1] = 1 
          
        #compare
        avg = 0
        valid = 0
        for i in range(scores.shape[0]):
          a1 = torch.where(adj_mat[i] == 1)[0]
          a2 = torch.where(scores[i] == 1)[0]
          # print(a1)
          # print(a2)
          a1=a1.cpu().detach().numpy()
          a2=a2.cpu().detach().numpy()
          x = len(np.intersect1d(a1, a2))
          y = len(torch.where(adj_mat[i] == 1)[0])       
          # print(f"x: {x}")
          # print(f"y: {y}")
          if y != 0:                           
            avg += x/y
            valid += 1
        avg = avg/valid
        return avg

Same as before

In [9]:
# Compute degree probabilities
# we must use train set
user_to_movie = adjacency_matrix_train.clone()
movie_to_user = adjacency_matrix_train.T
degrees = []
for user_idx in range(user_to_movie.shape[0]):
  degrees.append(len(torch.where(user_to_movie[user_idx] == 1)[0]))
for movie_idx in range(movie_to_user.shape[0]):
  degrees.append(len(torch.where(movie_to_user[movie_idx]==1)[0]))

Same as before

In [10]:
num_nodes = n_users+n_movies
adj_mat = torch.zeros(num_nodes, num_nodes)
for user_idx in range(n_users):
  adj_mat[user_idx][torch.where(user_to_movie[user_idx] == 1)[0] + n_users] = 1
for movie_idx in range(n_movies):
  adj_mat[n_users+movie_idx][torch.where(movie_to_user[movie_idx]==1)[0]] = 1

Same as before

In [ ]:
def compute_transition_probs(q):
  transition_prob = torch.zeros(num_nodes, num_nodes)
  for node_idx in range(num_nodes):
    # print(node_idx)
    transition_prob[node_idx] = torch.tensor([0]*num_nodes, dtype=torch.float64).to(device)
    valid_nodes = torch.where(adj_mat[node_idx] == 1)[0]
    transition_prob[node_idx][valid_nodes] = 1/q
  return transition_prob

Same as before

In [ ]:
def next_node(previous, current, transition_probs, p):
  node_indices = list(range(num_nodes))
  node_probs = transition_probs[current].clone()
  if previous != -1:
    node_probs[previous] = 1/p
  chosen_user = random.choices(node_indices, weights=node_probs, k=1)
  return chosen_user[0]

Mostly the same, except don't include start node as part of walk path

In [ ]:
import random
def random_walk(start_node, num_steps, transition_probs, p):
  walk = [start_node]
  while len(walk) < num_steps:
    current = walk[-1]
    previous = walk[-2] if len(walk) > 1 else -1
    next = next_node(previous, current, transition_probs, p)
    walk.append(next)
  return walk[1:]

Performm the optimizer.step() after each random walk. Essentially, use each random walk as a batch in training, as recommended by the prof. To help with time, iterate only over users. For each user, upweight dot product with good node embeddings by 200 and negatively sample K from the negative sample set. We want to optimize the same cost function as in part 1, so set loss to negative of objective.

In [ ]:
def train_loop(model, optimizer, num_steps, K, p, transition_probs):
  # rand_sample = random.sample(list(range(n_users)), 100)
  for user_idx in range(n_users):
    loss=0

    user_embed = model(torch.tensor(user_idx).to(device)).to(device)

    rand_walk = random_walk(user_idx, num_steps, transition_probs, p)

    rows_in_walk = torch.tensor(rand_walk).to(device)
    unique_rows_in_walk = torch.unique(rows_in_walk).to(device)
    
    rows = torch.arange(num_nodes).to(device)
    mask = torch.zeros_like(rows).to(device)
    mask[unique_rows_in_walk] = 1
    rows_notinwalk = rows[mask!=1]
    probs = torch.tensor(degrees)[mask!=1]

    path_node_embeds = model(rows_in_walk)
    path_scores = torch.matmul(user_embed, path_node_embeds.T)
    loss += 200*torch.sum(torch.nn.functional.logsigmoid(path_scores))

    neg_samples = random.choices(rows_notinwalk.tolist(), weights=probs.tolist(), k=K)
    neg_samples = torch.tensor(neg_samples).to(device)
    neg_embeds = model(neg_samples)
    bad_dps = torch.matmul(user_embed, neg_embeds.T).to(device)
    bad_logsigs = torch.nn.functional.logsigmoid(bad_dps).to(device)
    loss -= torch.sum(bad_logsigs)
    loss *= -1
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Same as before

In [ ]:
class EmbeddingLayer(nn.Module):
    def __init__(self):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(n, d)
        self.embedding.weight.data.uniform_(-1, 1)

    def forward(self, x):
        x = self.embedding(x)
        return x

model = EmbeddingLayer().to(device)
print(f"Model structure: {model}\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: EmbeddingLayer(
  (embedding): Embedding(10334, 10)
)

Layer: embedding.weight | Size: torch.Size([10334, 10]) | Values : tensor([[ 0.1018, -0.8347,  0.8218, -0.9378, -0.8945,  0.0339, -0.2602, -0.2440,
          0.3041,  0.0248],
        [-0.7408, -0.5972, -0.1294, -0.1780, -0.1342, -0.8037,  0.9305,  0.8798,
         -0.0501,  0.4738]], device='cuda:0', grad_fn=<SliceBackward0>) 



### Before doing a full grid search to determine best hyperparameters, lets demonstrate this gives better results for a specific hyperparameter setting

In [ ]:
model1 = EmbeddingLayer().to(device)

learning_rate = 1e-3
optimizer = torch.optim.SGD(model1.parameters(), lr=learning_rate) 
K = 100
p = 1
q = 5
num_steps = 20
transition_probs = compute_transition_probs(q)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model1, optimizer, num_steps, K, p, transition_probs)
    print(f"Train Recall150:\n{recall150(adjacency_matrix_train, model1)}")
    print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, model1)}")
print("Done!")

Epoch 1
-------------------------------
Train Recall150:
0.13217596109235916
Validation Recall150:
0.02418306927054637
Epoch 2
-------------------------------
Train Recall150:
0.26705664145118146
Validation Recall150:
0.055660989510014997
Epoch 3
-------------------------------
Train Recall150:
0.4330598391574411
Validation Recall150:
0.1587160823973081
Epoch 4
-------------------------------
Train Recall150:
0.5274406412784799
Validation Recall150:
0.22118816893836357
Epoch 5
-------------------------------
Train Recall150:
0.5732999966070091
Validation Recall150:
0.2525534027623959
Epoch 6
-------------------------------
Train Recall150:
0.5958534670066281
Validation Recall150:
0.2619393224829402
Epoch 7
-------------------------------
Train Recall150:
0.6101239153220269
Validation Recall150:
0.2725535568208503
Epoch 8
-------------------------------
Train Recall150:
0.6233384709479405
Validation Recall150:
0.28144317887680903
Epoch 9
-------------------------------
Train Recall150:


This works much better, with validation accuracy of approximately 28% at epoch = 10, instead of approximately 10% as we had in Training approach 1. This is expected because this new training approach is similar to training procedure in part 1 of the mini project, which gave similar scores(but better) on validation set since we directly learn embeddings instead of using random walks

(Note again here random walks that prefer BFS with length = 20 is closer towards directly learning, since we are more likely to just extract all of a start nodes neighbours in such a walk)

## Hyperparameter Tuning
We use grid search, with realistic p, q, K and num_walk values since these take a long time to run. Realistic here means values we ahve worked with in part 1, and when setting (p,q) since the important consideration is the ratio between them, making sure not to perform useless computations like testing p,q = 1,1 and 2,2, since these should theretically give same results. Furthermore, we know we want to do more BFS than DFS, so lets not try somthing like p,q = 5,1 or even 3,1, so that we dont waste computation to realize it performs poorly when we could have guessed it.

We will list all models out explicitly so that when our GPU usage terminates, we know where to pick up from

- (p,q) = (1,1), (1,2), (1,3), (1,4), (1,5), (2,1), (2,3), (3,4)
- num_walks = \[10, 20\]
- K = \[100, 200\]

In [ ]:
models = []
pqs = [(1,1), (1,2), (1,3), (1,4), (1,5), (2,1), (2,3), (3,4)]
walk_length = [10, 20]
Ks = [100, 200]

### This will take very long time to run.

In [ ]:
for pq in pqs:
  for walk_len in walk_length:
    for K in Ks:
      print(f'p = {pq[0]}, q = {pq[1]}, random walk length = {walk_len}, K = {K}')
      models.append(EmbeddingLayer().to(device))
      learning_rate = 1e-3
      optimizer = torch.optim.SGD(models[-1].parameters(), lr=learning_rate) 
      p = pq[0]
      q = pq[1]
      transition_probs = compute_transition_probs(q)

      epochs = 10
      for t in range(epochs):
          train_loop(models[-1], optimizer, walk_len, K, p, transition_probs)
      print(f"Train Recall150:\n{recall150(adjacency_matrix_train, models[-1])}")
      print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, models[-1])}")
      print("\n")

p = 1, q = 1, random walk length = 10, K = 100
Train Recall150:
0.6029852759174307
Validation Recall150:
0.28293956446099844


p = 1, q = 1, random walk length = 10, K = 200
Train Recall150:
0.582344533989359
Validation Recall150:
0.25674889731591377


p = 1, q = 1, random walk length = 20, K = 100
Train Recall150:
0.5923029135343069
Validation Recall150:
0.34947648328170533


p = 1, q = 1, random walk length = 20, K = 200
Train Recall150:
0.573443355514836
Validation Recall150:
0.3472896769883333


p = 1, q = 2, random walk length = 10, K = 100
Train Recall150:
0.6133295031913851
Validation Recall150:
0.2746546957267453


p = 1, q = 2, random walk length = 10, K = 200
Train Recall150:
0.5967981437857783
Validation Recall150:
0.2714026606147532


p = 1, q = 2, random walk length = 20, K = 100


In [ ]:
# From above observe:
# walk length = 20 better than walk length of 10
# K = 100 vs K=200 doesn't really matter but K=100 slightly better
models = []
pqs = [(1,1), (1,2), (1,3), (1,4), (1,5), (2,1), (2,3), (3,4)]
walk_len = 20
K = 100

for pq in pqs[1:]:
  print(f'p = {pq[0]}, q = {pq[1]}, random walk length = {walk_len}, K = {K}')
  models.append(EmbeddingLayer().to(device))
  learning_rate = 1e-3
  optimizer = torch.optim.SGD(models[-1].parameters(), lr=learning_rate) 
  p = pq[0]
  q = pq[1]
  transition_probs = compute_transition_probs(q)

  epochs = 10
  for t in range(epochs):
      train_loop(models[-1], optimizer, walk_len, K, p, transition_probs)
  print(f"Train Recall150:\n{recall150(adjacency_matrix_train, models[-1])}")
  print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, models[-1])}")
  print("\n")

p = 1, q = 2, random walk length = 20, K = 100
Train Recall150:
0.6190279024446358
Validation Recall150:
0.33847941518976943


p = 1, q = 3, random walk length = 20, K = 100
Train Recall150:
0.6284660624483009
Validation Recall150:
0.34146124733526406


p = 1, q = 4, random walk length = 20, K = 100
Train Recall150:
0.6333618345739782
Validation Recall150:
0.3124393212548879


p = 1, q = 5, random walk length = 20, K = 100


In [ ]:
# From above observe:
# walk length = 20 better than walk length of 10
# K = 100 vs K=200 doesn't really matter but K=100 slightly better
models = []
pqs = [(1,5), (2,1), (2,3), (3,4)]
walk_len = 20
K = 100

for pq in pqs:
  print(f'p = {pq[0]}, q = {pq[1]}, random walk length = {walk_len}, K = {K}')
  models.append(EmbeddingLayer().to(device))
  learning_rate = 1e-3
  optimizer = torch.optim.SGD(models[-1].parameters(), lr=learning_rate) 
  p = pq[0]
  q = pq[1]
  transition_probs = compute_transition_probs(q)

  epochs = 10
  for t in range(epochs):
      train_loop(models[-1], optimizer, walk_len, K, p, transition_probs)
  print(f"Train Recall150:\n{recall150(adjacency_matrix_train, models[-1])}")
  print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, models[-1])}")
  print("\n")

p = 1, q = 5, random walk length = 20, K = 100
Train Recall150:
0.6272568708134437
Validation Recall150:
0.33533926435024847


p = 2, q = 1, random walk length = 20, K = 100
Train Recall150:
0.5869295537782582
Validation Recall150:
0.3409530686685333


p = 2, q = 3, random walk length = 20, K = 100


In [ ]:
models = []
pqs = [(2,3), (3,4)]
walk_len = 20
K = 100

for pq in pqs:
  print(f'p = {pq[0]}, q = {pq[1]}, random walk length = {walk_len}, K = {K}')
  models.append(EmbeddingLayer().to(device))
  learning_rate = 1e-3
  optimizer = torch.optim.SGD(models[-1].parameters(), lr=learning_rate) 
  p = pq[0]
  q = pq[1]
  transition_probs = compute_transition_probs(q)

  epochs = 10
  for t in range(epochs):
      train_loop(models[-1], optimizer, walk_len, K, p, transition_probs)
  print(f"Train Recall150:\n{recall150(adjacency_matrix_train, models[-1])}")
  print(f"Validation Recall150:\n{recall150(adjacency_matrix_val, models[-1])}")
  print("\n")

p = 2, q = 3, random walk length = 20, K = 100
Train Recall150:
0.5955865939540852
Validation Recall150:
0.35492070078831495


p = 3, q = 4, random walk length = 20, K = 100


### We have yet to try (p,q) = (3,4) but this ratio lies between the others tried. Further, since the other ones all give around the same validation accuracy, we can just skip this since it will likely give same validation accuracy of around 34% and will take a long time to run.

In [36]:
print(f"Test Recall150:\n{recall150(adjacency_matrix_test, models[-1])}")

Test Recall150:
0.30356437823149856


# We are done

The main conclusion for this part is that using training approach 1, we get accuracy of about 10%, while training approach gives an accuracy of about 30%. In training approach one, we use precomputed walks, specifically we only store one walk and use it all the time. Furthermore, we use all nodes in training. 

In training approach 2, we use some improvements that makes the strategy similar to part 1. Specifically, we use a new walk each time,, and we only iterate over the users. This gives much better results, perhaps due to the simplicity in learning over 610 users rather that 610 + num_movies nodes. The node2vec paper uses precompute weights and all nodes, so strictly speaking we present our final accuracy as about 10% (model 8 on test set). However, the improvement is worthy of mentioning and experimenting with